In [ ]:
import pandas as pd
import geopandas as gpd

# Load hydrography data

In [ ]:
hydrography = gpd.read_file("zip://shp_water_dnr_hydrography.zip")

In [ ]:
hydrography.head()
for col in hydrography.columns:
    print(col)

In [ ]:
print(hydrography.loc[hydrography["wb_class"] == "Lake or Pond"])

# Load 7 county metro

In [ ]:
metro = gpd.read_file("zip://shp_bdry_metro_counties_and_ctus.zip")
metro.plot()

In [ ]:
metro.head()

In [ ]:
metro_dissolve = metro.dissolve(by = "CO_NAME")
metro_dissolve.plot()

In [ ]:
metro_dissolve.is_valid

# Clip hydro to metro

In [ ]:
# Varifying the projections
print(metro_dissolve.crs)
print(hydrography.crs)

In [ ]:
# Checking for invalid polygon geometries since the clip operation did not work
for i in hydrography.is_valid:
    if i == False:
        print("poly is false")

In [ ]:
# https://stackoverflow.com/questions/63955752/topologicalerror-the-operation-geosintersection-r-could-not-be-performed
hydro_valid = hydrography.buffer(0)

In [ ]:
# Checking for invalid polygon geometries since the clip operation did not work
for i in hydro_valid.is_valid:
    if i == False:
        print("poly is false")

In [ ]:
hydro_valid.head()

In [ ]:
hydro_clip = gpd.clip(hydro_valid, metro_dissolve)

In [ ]:
hydro_clip.plot()

In [ ]:
hydro_clip.head()

#hydro_lake = hydro_clip.loc[hydro_clip["wb_class"] == "Lake or Pond"]
#hydro_lake.plot()

# Load 2014, 2016, 2018 water files

In [ ]:
water2018 = gpd.read_file("zip://impaired_2018_lakes.zip")
water2016 = gpd.read_file("zip://impaired_2016_lakes.zip")
water2014 = gpd.read_file("zip://impaired_2014_lakes.zip")

In [ ]:
water2018.plot()

In [ ]:
for col in water2018.columns:
    print(col)

In [ ]:
water2018 = water2018.drop(["CAT", "CAT_DESC", "REACH_DESC", "AREA_ACRES", "AFFECTED_U", "LIKE_MEET", "NON_POLL", 
                "NAT_BACK", "ADD_MON", "APPROVED", "NEEDS_PLN", "NEW_IMPAIR", "HUC_8", "HUC_8_NAME", "HUC_4", "BASIN", 
                "COUNTY", "TRIBAL_INT", "INDIAN_RES", "AMMONIA", "CHLORIDE", "FISHESBIO", "HG_F", "HG_W", 
                "NUTRIENTS", "PCB_F", "PFOS_F", "Shape_Leng", "Shape_Area"], axis = 1)


In [ ]:
water2018

In [ ]:
for col in water2016.columns:
    print(col)

In [ ]:
water2016.drop(["CAT", "DATASET_NA", "REACH_DESC", "AREA_ACRES", "AFFECTED_U", "TMDL_NOT_R", "TMDL_NOT_1", "IMPAIR_P_1", "NEW_IMPAIR", "NEW_IMPA_1", 
                "TMDL_APPRO", "TMDL_APP_1", "TMDL_NEEDE", "TMDL_NEE_1", "HUC_8", "HUC_8_NAME", "HUC_4", 
                "BASIN", "COUNTY", "TRIBAL_INT", "INDIAN_RES", "CHLORIDE", "FISHESBIO", "HG_F", "HG_W", 
                "NUTRIENTS", "PCB_F", "PFOS_F", "SHAPE_Leng", "SHAPE_Area"], axis = 1)

In [ ]:
for col in water2014.columns:
    print(col)

In [ ]:
water2014 = water2014.drop(["LOCATION", "ACRES", "CAT", "AFFECTED_U", "NOPLN", "APPROVED", "NEEDSPLN", "NEW_2014", 
                "HUC8", "HUC8_NAME", "HUC4", "BASIN", "ALL_COUNTI", "WDWMO_NAME", "WDWMO_TYPE", "Chloride", 
                "HgF", "HgW", "Nutrients", "PCBF", "PFOS_W", "SHAPE_Leng", "Shape_Le_1", "Shape_Area"], axis = 1)
water2014

In [ ]:
water2014.rename(columns = {"WATER_NAME" : "NAME"})

# Load water 2020 data csv

In [ ]:
water2020 = gpd.read_file("wq-iw1-65.csv")

In [ ]:
for col in water2020.columns:
    print(col)

In [ ]:
water2020.head()
test = water2020[["Water body name", "AUID", "Water body type", "Use Class", "Pollutant or stressor", "geometry"]]
test

In [ ]:
test2 = test.loc[(test["Water body type"] == "Lake")]
test2

# Trying to remove duplicates

In [ ]:
#join test2 to water2018
# output 1946 rows...there are duplicates...???
jointest = test2.merge(water2018, on = "AUID")

jointest

In [ ]:
test3 = test2.groupby("AUID", as_index = False)["Pollutant or stressor"].apply(";".join)
test3

In [ ]:
#join test3 back to test2

test4 = test2.merge(test3, how = "inner", on = "AUID")
test4

# Load blockgroup data and intersecting it with the water data

In [ ]:
blockgroups_df = gpd.read_file('zip://tl_2019_27_bg.zip')
print(f'Loaded {len(blockgroups_df):,} block groups')

In [ ]:
blockgroups_df.plot()

In [ ]:
print(blockgroups_df.crs)
print(water2018.crs)

In [ ]:
bg_proj = blockgroups_df.to_crs('EPSG:26915')
bg_proj.plot()

In [ ]:
water2018_intersect_bg_proj = gpd.overlay(water2018, bg_proj, how='intersection')
water2018_intersect_bg_proj.plot()
water2018_intersect_bg_proj.head()

In [ ]:
### Code Review Comments ###
# add columns and then remove duplicates again
# find lakes that have been added/removed, and look at the traffic at that spot now
# Smaller scale? Just one city
# function for selecting data with block groups